In [1]:
import pandas as pd

# read the entire file into a python array
filename = "yelp_academic_dataset_checkin.json"
with open(filename, 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"
# now, load it into pandas
data_df = pd.read_json(data_json_str)
data_df = data_df[:20000]
# Get time distribution in one week and a day
week_time_distribution = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
day_time_distribution = [str(i) for i in range(24)]
column_names = week_time_distribution + day_time_distribution
column_names.insert(0, "business_id")
business_attributes = pd.DataFrame(columns = column_names)
week_time_distribution = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
day_time_distribution = [str(i) for i in range(24)]
column_names = week_time_distribution + day_time_distribution
column_names.insert(0, "business_id")
business_attributes = pd.DataFrame(columns = column_names)
for index, row in data_df.iterrows():
    time_dic = {column_names[i]:0 for i in range(1, len(column_names))}
    time_list = row['time']
#     print time_dic, column_names[1:]
    for time in time_list:
        hour = 0
        check_ins = 0
        date = time[:3]
        for i in range(5, len(time)):
            if time[i] == ':':
                hour = time[4 : i]
                count_ins = time[i + 1:]
                break
        time_dic[str(hour)] = count_ins
        time_dic[str(date)] = count_ins
    time_dic['business_id'] = row['business_id']
    result = [time_dic[name] for name in column_names]
#     print time_dic, result
    business_attributes.loc[index] = result

In [2]:
filename = "yelp_academic_dataset_business.json"
with open(filename, 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"
# now, load it into pandas
data_df = pd.read_json(data_json_str)
# business_category = pd.DataFrame([data_df['business_id'], data_df['categories']])
business_category = data_df.ix[:,['business_id', 'categories']]
# print business_category.head()

In [36]:
business_time_category = business_attributes.merge(business_category, on = 'business_id', how = 'inner')
business_categories = []
business_category_count = {}
for index, row in business_time_category.iterrows():
    business_category_list = row['categories']
#     print business_category_list
    if business_category_list:
        for category in business_category_list:
            if category not in business_category_count:
                business_category_count[category] = 0
            else:
                business_category_count[category] += 1
for busi in business_category_count:
    if business_category_count[busi] > 2000:
        business_categories.append(busi)
print business_categories
business_time_category['in_categories'] = False
for index, row in business_time_category.iterrows():
    categories = row['categories']
    if categories:
        for category in categories:
            if category in business_categories:
                business_time_category.set_value(index, 'in_categories', True)

[u'Restaurants', u'Nightlife', u'Food', u'Shopping']


In [44]:
business_category_detail = business_time_category[business_time_category['in_categories'] == True]
for category in business_categories:
    business_category_detail[category] = 0
for index, row in business_category_detail.iterrows():
    categories = row['categories']
    for category in categories:
        if category in business_categories:
            business_category_detail.set_value(index, category, 1)
business_category_detail.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,business_id,Mon,Tue,Wed,Thu,Fri,Sat,Sun,0,1,...,20,21,22,23,categories,in_categories,Restaurants,Nightlife,Food,Shopping
0,7KPBkxAOEtb3QeIL9PEErg,1,1,1,2,1,3,1,2,1,...,2,1,1,1,"[Automotive, Convenience Stores, Food, Gas & S...",True,0,0,1,0
3,nhZ1HGWD8lMErdn3FuWuTQ,1,3,1,1,1,2,1,1,1,...,1,1,1,3,"[American (Traditional), Sports Bars, American...",True,1,1,0,0
6,BH0K6SxPnY3wTRB7tA27hQ,0,2,2,1,1,1,2,1,2,...,0,0,1,2,"[Chinese, Restaurants]",True,1,0,0,0
7,dPxL6P_Gp74CqSPuL59Srg,1,1,1,1,1,1,1,1,0,...,1,1,1,1,"[American (Traditional), Restaurants]",True,1,0,0,0
10,HRIUU2Z4EsZBo59P0I2ZYA,2,1,2,0,1,1,1,1,1,...,1,0,1,2,"[Chicken Wings, Pizza, Italian, Restaurants]",True,1,0,0,0


In [52]:
from sklearn import svm
time_distribution = business_category_detail.ix[:,1:32]
is_restaurant = business_category_detail['Restaurants']
clf = svm.SVC()
clf.fit(time_distribution, is_restaurant)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [77]:
from sklearn import svm
from sklearn.decomposition import PCA
for category in business_categories:
    components = [2, 5, 10, 15]
    precision = 0
    for n in components:
        pca = PCA(n_components = n)
        time_distribution = business_category_detail.ix[:,1:32]
#         time_distribution = pca.fit(time_distribution)
        is_restaurant = business_category_detail[category]
        clf = svm.SVC()
        train_data = time_distribution.ix[:15000, :]
        train_data = pca.fit_transform(train_data)
        train_target= is_restaurant.ix[:15000]
        validation_data = time_distribution.ix[15000:, :]
        validation_data = pca.fit_transform(validation_data)
        validation_target = is_restaurant.ix[15000:].tolist()
        # print train_target
        # print len(validation_data)
        clf.fit(train_data, train_target)
        result = clf.predict(validation_data)
        right_number, false_number = 0, 0
        for i in range(len(result)):
            if result[i] == validation_target[i]:
                right_number += 1
            else:
                false_number += 1
        if right_number/float(len(result)) > precision:
            precision = right_number/float(len(result))
    print category + ":  " + str(precision)

Restaurants:  0.574535050072
Nightlife:  0.836909871245
Food:  0.780829756795
Shopping:  0.788555078684


In [ ]:
import pandas as pd
filename = "yelp_academic_dataset_review.json"
with open(filename, 'rb') as f:
    data = f.readlines()
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"
# now, load it into pandas
data_df = pd.read_json(data_json_str, engine = 'python')

In [1]:
import pandas as pd
import json
review_data = []
review_data_dic = []
count = 0
filename = "yelp_academic_dataset_review.json"
with open(filename, 'rb') as f:
    for line in f:
        review_data.append(line)
        count += 1
        if count == 1000000:
            break
for review in review_data:
    review = json.loads(review)
    review_info = {}
    review_info['business_id'] = review['business_id']
    review_info['user_id'] = review['user_id']
    review_data_dic.append(review_info)
review_data_pandas = pd.DataFrame(review_data_dic)
print review_data_pandas.head()

              business_id                 user_id
0  2aFiy99vNLklCx3T_tGS9A  KpkOkG6RIf4Ra25Lhhxf1A
1  2aFiy99vNLklCx3T_tGS9A  bQ7fQq1otn9hKX-gXRsrgA
2  2aFiy99vNLklCx3T_tGS9A  r1NUhdNmL6yU9Bn-Yx6FTw
3  2LfIuF3_sX6uwe-IR-P0jQ  aW3ix1KNZAvoM8q-WghA3Q
4  2LfIuF3_sX6uwe-IR-P0jQ  YOo-Cip8HqvKp_p9nEGphw


In [2]:
user_review_count = {}
for index, row in review_data_pandas.iterrows():
    if row['user_id'] in user_review_count:
        user_review_count[row['user_id']] += 1
    else:
        user_review_count[row['user_id']] = 1
sort_user_review_counts = sorted(user_review_count.items(), key=lambda x: x[1], reverse = True)

In [3]:
business_review_count = {}
for index, row in review_data_pandas.iterrows():
    if row['business_id'] in business_review_count:
        business_review_count[row['business_id']] += 1
    else:
        business_review_count[row['business_id']] = 1
sort_business_review_counts = sorted(business_review_count.items(), key=lambda x: x[1], reverse = True)

In [4]:
business_user_count = {}
for index, row in review_data_pandas.iterrows():
    user_business = row['user_id'] + row['business_id']
    if user_business in business_user_count:
        business_user_count[user_business] += 1
    else:
        business_user_count[user_business] = 1
sort_business_user_counts = sorted(business_user_count.items(), key=lambda x: x[1], reverse = True)

In [28]:
frequent_users = [sort_user_review_counts[i][0] for i in range(5000)]
# print frequent_users

In [36]:
business_users = {}
frequent_users = [sort_user_review_counts[i][0] for i in range(5000)]
for index, row in review_data_pandas.iterrows():
    if row['user_id'] in frequent_users:
        if row['business_id'] in business_users:
            business_users[row['business_id']].append(row['user_id'])
        else:
            business_users[row['business_id']] = [row['user_id']]
    else:
        if row['business_id'] not in business_users:
            business_users[row['business_id']] = []

In [ ]:
import pandas as pd
import json
review_data = []
review_data_dic = []
count = 0
filename = "yelp_academic_dataset_review.json"
with open(filename, 'rb') as f:
    for line in f:
        review_data.append(line)
        count += 1
        if count == 1000000:
            break
for review in review_data:
    review = json.loads(review)
    review_info = {}
    review_info['business_id'] = review['business_id']
    review_info['user_id'] = review['user_id']
    review_data_dic.append(review_info)
review_data_pandas = pd.DataFrame(review_data_dic)
print review_data_pandas.head()

user_review_count = {}
for index, row in review_data_pandas.iterrows():
    if row['user_id'] in user_review_count:
        user_review_count[row['user_id']] += 1
    else:
        user_review_count[row['user_id']] = 1
sort_user_review_counts = sorted(user_review_count.items(), key=lambda x: x[1], reverse = True)

business_review_count = {}
for index, row in review_data_pandas.iterrows():
    if row['business_id'] in business_review_count:
        business_review_count[row['business_id']] += 1
    else:
        business_review_count[row['business_id']] = 1
sort_business_review_counts = sorted(business_review_count.items(), key=lambda x: x[1], reverse = True)

business_user_count = {}
for index, row in review_data_pandas.iterrows():
    user_business = row['user_id'] + row['business_id']
    if user_business in business_user_count:
        business_user_count[user_business] += 1
    else:
        business_user_count[user_business] = 1
sort_business_user_counts = sorted(business_user_count.items(), key=lambda x: x[1], reverse = True)

frequent_users = [sort_user_review_counts[i][0] for i in range(5000)]

business_users = {}
frequent_users = [sort_user_review_counts[i][0] for i in range(5000)]
for index, row in review_data_pandas.iterrows():
    if row['user_id'] in frequent_users:
        if row['business_id'] in business_users:
            business_users[row['business_id']].append(row['user_id'])
        else:
            business_users[row['business_id']] = [row['user_id']]
    else:
        if row['business_id'] not in business_users:
            business_users[row['business_id']] = []

              business_id                 user_id
0  2aFiy99vNLklCx3T_tGS9A  KpkOkG6RIf4Ra25Lhhxf1A
1  2aFiy99vNLklCx3T_tGS9A  bQ7fQq1otn9hKX-gXRsrgA
2  2aFiy99vNLklCx3T_tGS9A  r1NUhdNmL6yU9Bn-Yx6FTw
3  2LfIuF3_sX6uwe-IR-P0jQ  aW3ix1KNZAvoM8q-WghA3Q
4  2LfIuF3_sX6uwe-IR-P0jQ  YOo-Cip8HqvKp_p9nEGphw


In [4]:
# np.savetxt(r'business_user_review_network_5000', df.values, fmt='%d')
business_user_review_network.to_pickle('business_user_review_network_5000')

In [1]:
import pandas as pd
business_users_info = pd.read_pickle('business_user_review_network_5000')
filename = "yelp_academic_dataset_business.json"
with open(filename, 'rb') as f:
    data = f.readlines()

data = map(lambda x: x.rstrip(), data)

data_json_str = "[" + ','.join(data) + "]"
data_df = pd.read_json(data_json_str)
business_category = data_df.ix[:,['business_id', 'categories']]

business_users_category = business_users_info.merge(business_category, how = 'inner')

category_count = {}
for index, busi in business_users_category.iterrows():
    categories = busi['categories']
    for category in categories:
        if category in category_count:
            category_count[category] += 1
        else:
            category_count[category] = 1
sort_business_category_counts = sorted(category_count.items(), key=lambda x: x[1], reverse = True)

frequent_category_list = [item[0] for item in sort_business_category_counts[:4]]

for category in frequent_category_list:
    business_users_category[category] = False
    
for index in range(5000):
    row = business_users_category.iloc[[index]]
    categories = row['categories'].tolist()
    for category in categories[0]:
        if category in frequent_category_list:
            business_users_category.set_value(index, category, True)

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.decomposition import PCA
for busi in frequent_category_list:
    maxPrecision = 0
    for n in [5, 20, 50, 100]:
        for k in [5, 10, 20, 50]:
            pca = PCA(n_components = n)
            knn = KNeighborsClassifier(n_neighbors = 10)
            train_x = pca.fit_transform(business_users_category.ix[:4500, 1:5000])
            train_y = business_users_category.ix[:4500][busi].tolist()
            test_x = pca.fit_transform(business_users_category.ix[500:,1:5000])
            test_y = business_users_category.ix[500:][busi].tolist()
            knn.fit(train_x, train_y)
            # print train_y
            predictions = knn.predict(test_x)
            right_count = 0
            for i in range(500):
                if predictions[i] == test_y[i]:
                    right_count += 1
            if right_count / 500.0 > maxPrecision:
                maxPrecision = right_count / 500.0
                print right_count
    print "Presision for %s is %.2f ." %(busi, maxPrecision)

356
Presision for Restaurants is 0.71 .
387
388
391
Presision for Food is 0.78 .
398
399
401
Presision for Nightlife is 0.80 .
409
410
411
Presision for Bars is 0.82 .
